In [2]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [4]:
import openai

In [5]:
from langchain.agents import *
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor

## Connect with the database

In [6]:
!pip install pymysql

In [23]:
db_user = 'root'
db_password = '6e6b697565'
db_host = 'localhost'
db_name = 'bigyan_database'
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

### Setup the LLM, toolkit and agen executer

In [24]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [25]:
# create agent
toolkit = SQLDatabaseToolkit(db= db, llm = llm)

In [28]:
agen_executor = create_sql_agent(
    llm= llm,
    toolkit=toolkit,
    handling_parsing_errors = True,
    verbose = True
)

In [29]:
agen_executor.run('How many tables do we have?')



> Entering new SQL Agent Executor chain...
I should first find out the number of tables in the database before proceeding with any specific queries.
Action: sql_db_list_tables
Action Input: smartwatchesI need to check the number of tables in the database.
Action: sql_db_list_tables
Action Input: smartwatchesThere is only one table in the database.
Final Answer: 1 table.

> Finished chain.


'1 table.'

In [30]:
agen_executor.run('how many rows do we have and list them all.')



> Entering new SQL Agent Executor chain...
We need to find out the number of rows and get a list of all the rows in the database table.
Action: sql_db_query_checker
Action Input: SELECT COUNT(*) FROM table_name;SELECT COUNT(*) FROM table_name;We need to retrieve the actual data as well, not just the count.
Action: sql_db_query
Action Input: SELECT * FROM table_name;Error: (pymysql.err.ProgrammingError) (1146, "Table 'bigyan_database.table_name' doesn't exist")
[SQL: SELECT * FROM table_name;]
(Background on this error at: https://sqlalche.me/e/20/f405)We need to check the available tables in the database.
Action: sql_db_list_tables
Action Input: smartwatchesThe table we are looking for is named "smartwatches."
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM smartwatches;[(287,)]There are 287 rows in the "smartwatches" table.
Final Answer: There are 287 rows in the "smartwatches" table.

> Finished chain.


'There are 287 rows in the "smartwatches" table.'

### Let's see the results with some complex commands

In [31]:
agen_executor.run('Extract Brands of smartwatches with CurrentPrice above 10000.')



> Entering new SQL Agent Executor chain...
We need to find the table that contains information about smartwatches and their prices, then filter for those with a CurrentPrice above 10000 and extract the corresponding Brands.
Action: sql_db_list_tables
Action Input: smartwatchesWe have a table named "smartwatches" that likely contains the information we need.
Action: sql_db_schema
Action Input: smartwatches
CREATE TABLE smartwatches (
	`MyUnknownColumn` INTEGER, 
	`Brand` TEXT, 
	`Current Price` INTEGER, 
	`Original Price` INTEGER, 
	`Discount Percentage` DOUBLE, 
	`Rating` DOUBLE, 
	`Number OF Ratings` INTEGER, 
	`Model Name` TEXT, 
	`Dial Shape` TEXT, 
	`Strap Color` TEXT, 
	`Strap Material` TEXT, 
	`Touchscreen` TEXT, 
	`Battery Life (Days)` DOUBLE, 
	`Bluetooth` TEXT, 
	`Display Size` TEXT, 
	`Weight` TEXT
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from smartwatches table:
MyUnknownColumn	Brand	Current Price	Original Price	Discount Percentage	Ratin

'Brands of smartwatches with CurrentPrice above 10000 are noise, honor, samsung, garmin, apple, fitbit, amazfit, huawei, and fossil.'